# CH2. Finding Time series data
_Practical Time Seriese Data Analysis 의 2장 2절. 번역이 매끄럽지 못한 부분은 이해하는 대로 약간의 의역을 담아 노트필기를 해본다._

---

시간축으로 구성되어있는 일반적인 시계열 데이터 뿐만 아니라, 직접 수집한 시계열 데이터를 저자는 found-time-series 라고 일컫었다.  
발견된 시계열이라고 부른 이것은, 시계열 분석을 위한 별도의 비용(단계라고 생각함)없이 기록한 개별 데이터를 모아둔 것이다. 말그대로 시계열을 위한 데이터가 아니지만, 시계열을 구성할 수 있는 데이터를 의미한다.

보통 데이터베이스 스키마를 설계할 때 많은 기관에서 시계열 분석을 고려하지 않는 경우가 많다. 이러한 경우 서로 다른 테이블과 자료로부터 시계열을 수집하고 구성할 필요가 있다.



##### 1. 타임스탬프가 찍힌 이벤트 레코드  
데이터에 타임스탬프가 존재한다면 시계열이 잠재적으로 구성될 가능성이 있다. 파일에 접근한 시간을 기록하지만 해도 시계열을 구성할 수 있다. 예를 들어, 임의의 시점과 그 시점을 기준으로 이후의 시점의 타임스탬프를 이용하여 시간의 변화량을 모델링하여, **시간에 대한 시간축과 시간 변화량에 대한 값축으로 시계열을 구성할 수 있다** (기간을 주제로 시계열 데이터를 구성할 수 있다는 말). 추가로 변화량의 일부를 합하여 더 긴 기간에 대한 평균이나 총합을 구하거나, 각각을 개별적으로 구할 수도 있다.
##### 2. 시간을 대체하는 '시간이 없는' 측정 데이터
데이터가 명시적으로, 우리가 생각하는 '시간'을 포함하지는 않지만, 데이터 셋 안에서 논리적 흐름에 따라 시간이라는 개념이 설명되는 경우를 말한다. 예를 들어, 특정 비율(확률, 혹은 주기 같은 것을 말하는 듯)에 따라 센서가 수축(이완)하듯이, 어떤 실험 매개변수에 의해 간격이 발생하면 '간격 대 값'으로 데이터를 생각해볼 수 있다. 즉, **변수 중 하나가 의미적으로 시간에 대응할 수 있는 개념이라면 시계열이 있다고 본다**. 스펙트럼의 파장과 같이 축 중 하나가 간격과 순서와 같은 명시적인 관계를 가진다면, 시계열 데이터가 존재한다고 볼 수 있다.
##### 3. 물리적 흔적
의학, 청작학, 기상학 등과 같은 많은 과학 학문 분야에서는 물리적 흔적을 기록한다. 과거에는 아날로그식 처리로 수집, 기록하였지만 현대에는 디지털 형태로 저장한다. 이러한 물리적 흔적은 이미지 파일이나 데이터베이스에 한 필드의 단일 벡터와 같은 형태로 저장된다. 따라서 명백히 시계열이라고 말하긴 어렵지만, 이또한 여전히 시계열이다.  
_(특정 크기 행렬 혹은 벡터로 저장된 이미지 파일 같은 것을 생각해볼 수 있을 것 같다. 특정 부위의 종양이 점차 호전되는 형상 데이터라던지, 특정 위경도의 기압 기록 그리고 그 변화, 아니면 특정 기상 상태의 강도가 어떻게 변화하는지 같은 것, 혹은 어떤 특정 공간안의 이벤트 같은 것들이 시간의 흐름에 따라 축적되는 상태라고 생각한다. 일정한 기간에 따라 지속적으로 수집된 데이터면 당연히~(?)~ 시계열이겠지만, 그냥 기록 자체가 지속적으로 쌓이기만 해도 그것은 특정 대상의 시계열 데이터라고 볼 수 있는 것이라는 의미로 생각된다.)_
    

##### 데이터 재구성에 다음과 같은 '시계열에 특화된 기법'을 사용한다.
1. 해결하고자 하는 문제에 맞는 형태로 데이터의 간격을 교정한다.  
종종 데이터는 필요 이상으로 구체적인 시간 정보를 제공한다.  
2. 사전 관찰을 피하기 위해  
가용 데이터를 생산하는 타임스탬프를 데이터에 사용하지 않는 방법을 이해한다.  
3. 관련된 모든 기간 기록  
아무 일도 일어나지 않았더라도, 0으로 채우는 일이 있더라도 다른 정보들처럼 빈 기간 또한 활용해야한다.  
4. 사전관찰을 피하기 위해  
아직 알아서는 안되는 정보를 생산하는(예측의 타겟이 되어야할 상태가 컬럼으로 사용되는) 타임스탬프를 데이터에 사용하지 않는 것을 이해해야 한다.


    

# 2.2 테이블 집합에서 시계열 데이터 집합 개선하기
---
'found time series는 sql DB에 저장된 상태 및 이벤트형 데이터로부터 추출된 데이터이다. 구조화된 데이터베이스에 대량의 데이터가 지속적으로 저장되므로 매우 적절한 예라고 볼 수 있다'고 말한다.

예시 데이터는 비영리 단체에서의 회원 가입연도와 등급, 주간 메일 확인 횟수, 기부 타임스탬프와 금액으로 이루어져 있다.



## libraries and data load

In [1]:
# 라이브러리 모셔오기

from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [2]:
# 데이터 모셔오기

df_yearjoined = pd.read_csv('/Users/Angela/Desktop/Personal/TimeSeriesAnalysis/BookRepo-master/Ch02/data/year_joined.csv')
df_yearjoined.head(10)

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018
5,5,bronze,2017
6,6,inactive,2016
7,7,silver,2018
8,8,inactive,2017
9,9,silver,2016


In [3]:
df_email = pd.read_csv('/Users/Angela/Desktop/Personal/TimeSeriesAnalysis/BookRepo-master/Ch02/data/emails.csv')
df_email.head(10)

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00
5,2.0,1.0,2015-08-10 00:00:00
6,1.0,1.0,2015-08-24 00:00:00
7,2.0,1.0,2015-08-31 00:00:00
8,3.0,1.0,2015-09-07 00:00:00
9,2.0,1.0,2015-09-14 00:00:00


In [4]:
df_dona = pd.read_csv('/Users/Angela/Desktop/Personal/TimeSeriesAnalysis/BookRepo-master/Ch02/data/donations.csv')
df_dona.head(10)

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0
5,75.0,2017-01-23 12:55:47,1.0
6,50.0,2016-05-05 19:03:13,1.0
7,50.0,2017-07-25 11:39:21,2.0
8,25.0,2016-06-07 21:50:09,2.0
9,25.0,2016-03-15 12:04:14,2.0


In [5]:
# data type replace

df_email['user'] = df_email['user'].astype('int64')
df_email['emailsOpened'] = df_email['emailsOpened'].astype('int64')
df_dona['user'] = df_dona['user'].astype('int64')

In [6]:
df_email.head(3)

,emailsOpened,user,week
0,3,1,2015-06-29 00:00:00
1,2,1,2015-07-13 00:00:00
2,2,1,2015-07-20 00:00:00


In [7]:
df_dona.head(3)

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0
1,50.0,2015-08-25 19:01:45,0
2,25.0,2015-03-26 12:03:47,0


In [8]:
# data type replace (time)

import datetime

df_email['week'] = pd.to_datetime(df_email['week'])
df_dona['timestamp'] = pd.to_datetime(df_dona['timestamp'])

In [9]:
df_email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25488 entries, 0 to 25487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   emailsOpened  25488 non-null  int64         
 1   user          25488 non-null  int64         
 2   week          25488 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 597.5 KB


## question

이런 질문이 있다고 가정한다.  
1. 시간이 지나면서 바뀌는 수신자의 메일 대응: 메일을 읽는가, 읽지 않는가?
2. 멤버십 내역: 회원이 회원 자격을 상실한 기간이 있는가?
3. 거래 내역: 소비자가 특정 항목을 언제 구매할지 예측할 수 있는가?

이 데이터들을 시계열 문제로 해석하여 approach 한다면 다음과 같다.
1. 메일 응답 회원과 회원별 응답 시간(기부까지 걸린 시간) 관계를 2D histogram으로 만들어서 메일 수신에 따른 회원의 피로감을 추측한다.
2. 기부 예측을 시계열 예측 문제로 전환한다.
3. 회원의 행동 자취에 대한 패턴 유무를 검사한다. 이메일을 연달아 지우는 행동(이게 수집이 가능한가? 그저 확인하지 않은 것을 의미하는 건가?)처럼 회원이 조직을 떠날 것으로 알려주는 이벤트의 전형적 패턴이 있을까? 시계열 분석은 외부에 드러난 행동을 통해 특정 회원의 내재된 상태를 감지하고 표현할 수 있다.

이 데이터들의 의문점을 정리하면 다음과 같다.
1. `_yearjoined` 테이블에서 회원의 상태는 회원이 가입한 연도와 가입 당시의 상태일 수도 있고, 현재의 상태일 수도 있다.  
    이 부분은 과거 데이터 분석에 회원 상태는 현재를 입력하게 되면 시계열 모델에 미래를 알려주는 것이기 때문에 '사전관찰(look-ahead)'로 볼 수 있다. 
    이렇게 **값이 언제 할당되었는지 알 수 없는 상태 변수의 사용은 지양해야 한다.**
2. `_email` 테이블에서의 기간은 특정 시점을 알 수는 없으나, 해당 주간의 집계를 기록한 것이다. 한 주 간격으로 발생한 이벤트(이메일 열람같은 것)라고 보기보다는 매주의 기간에 대한 것으로 받아들여야 한다. 
    **주가 나누어져 있다면, 토요일이나 월요일, 일요일 등 시작이 일정하도록 정해져있도록 되어있는지 확인하는 것도 중요하다.**
    또한 결측이 발생한 주가 있는지도 확인해야 한다. 특정 회원이 한번도 이메일을 열람하지 않은 상태같은 것이 예시이다. **이 경우에는 null이 항상 존재해야 하는데, 아무런 일도 일어나지 않은 주 또한 데이터의 일부이기 때문이다.**
    



## 2.2.1 작업의 예: 시계열 데이터 집합 조립하기

연관된 데이터를 사용할 수 있다면, 서로 다른 타임스탬프 방식과 데이터 세분화의 수준을 고려하여 데이터 목록을 정렬할 수 있다. 

yearjoined table: 연간 회원 상태  
email: 이메일 열람에 대한 주간 누적 기록
donation: 기부한 순간의 타임스탬프

## 의문 1을 해결하기 위한 데이터 확인

In [10]:
# 1000명의 모든 회원이 단 하나의 상태만을 가진다는 의미: 즉 가입연도 + 현재의 상태(변화해온 상태가 아닌)를 나타낼 가능성이 높음.

df_yearjoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


## 의문2를 해결하기 위한 데이터 확인

In [11]:
# 이메일 오픈수가 1 이하인 데이터 = 0개

df_email[df_email.emailsOpened < 1]

,emailsOpened,user,week


null이 아예 발생하지 않았거나, 모든 회원이 이메일을 열람했다는 이벤트가 적어도 하나 존재한다는 두 가지 가능성이 있다.

이메일 데이터프레임에서 유저들을 이메일 오픈한 기록이 있는 주간의 횟수로 그룹바이한 후 그 user들이 몇이나 있는지 보았다.

In [12]:
# 이메일 열어본 기록이 있는 회원들의 명수는 539명이었다.

df_email.groupby('user')[['emailsOpened']].count()

,emailsOpened
user,
1,139
3,8
5,46
6,58
9,89
...,...
991,1
992,3
993,38


모든 회원이 매주 한 번은 이메일을 열어본다는 가능성은 희박하기 때문에 특정 한 회원의 기록을 살펴봄으로써 어떤 것인지 판단해본다.

아래를 보면 일부 주가 누락된 것을 확인할 수 있다. (2017-12-18 ~ 2018-01-01 사이의 기록)



In [13]:
df_email[df_email['user'] == 998].sort_values('week')

,emailsOpened,user,week
25464,1,998,2017-12-04
25465,3,998,2017-12-11
25466,3,998,2017-12-18
25467,3,998,2018-01-01
25468,3,998,2018-01-08
25469,2,998,2018-01-15
25470,3,998,2018-01-22
25471,2,998,2018-01-29
25472,3,998,2018-02-05
25473,3,998,2018-02-12


혹은 사건이 처음 발생한 시점과 마지막 발생 시점 사이에 주가 몇 개 기록되었는지 계산하는 방식으로도 검증가능하다.

In [14]:
# (마지막 주 - 처음 주)의 일수를 7로 나눈 것

( max(df_email[df_email['user'] == 998]['week']) - min(df_email[df_email['user'] == 998]['week']) ).days / 7

25.0

In [15]:
# 해당 회원의 주간 데이터 개수

df_email[df_email['user'] == 998].shape

(24, 3)

모든 주가 포함되려면 25+1(마지막 오프셋. 누락된 시작 날짜.) 26주가 있어야 하지만 24개의 주 기록만 있는 것으로 확인되었다.

누락된 주가 있다는 것을 보정해본다. 그 부분을 0으로 채워넣도록 하는 형태이다.

### 기록이 없는 주간 보정

기록된 날짜의 가장 min보다도 이전이거나, 혹은 가장 마지막인 max보다도 이후에 사건이 발생할 수 있기 때문에 누락된 모든 주를 식별하는 것은 어려운 일이다. 

이때는 널값이 아닌, **사건이 기록된 회원의 데이터 중에서 처음과 마지막 시기의 사잇값으로 누락된 기간을 채울 수 있다**.

1. 판다스의 색인 기능을 활용하면 모든 회원에 대해 누락된 주를 쉽게 넣을 수 있다. 다중 색인을 위한 MultiIndex를 만들 수 있는데, 모든 회원과 주에 대한 모든 조합을 생성한다.  
다시 말해, 피처간의 곱집합(cartesian product)을 수행한다.


In [16]:
complete_idx = pd.MultiIndex.from_product((set(df_email['week']), set(df_email['user'])))

In [17]:
complete_idx

MultiIndex([('2016-07-11',   1),
            ('2016-07-11',   3),
            ('2016-07-11',   5),
            ('2016-07-11',   6),
            ('2016-07-11',   9),
            ('2016-07-11',  10),
            ('2016-07-11',  14),
            ('2016-07-11',  16),
            ('2016-07-11',  20),
            ('2016-07-11',  21),
            ...
            ('2015-04-13', 973),
            ('2015-04-13', 977),
            ('2015-04-13', 982),
            ('2015-04-13', 984),
            ('2015-04-13', 987),
            ('2015-04-13', 991),
            ('2015-04-13', 992),
            ('2015-04-13', 993),
            ('2015-04-13', 995),
            ('2015-04-13', 998)],
           length=93247)

2. 테이블을 아래 complete_idx로 채워 넣는다. 이를 인덱스로 설정해주고, 이때 값에는 기록할 것이 없다는 의미로 0을 채워넣도록 한다.

3. 또한 인덱스를 재설정하여, 인덱스로 사용된 회원과 주의 정보를 다시 컬럼으로 만든다. 이때 회원과 주간의 컬럼 이름이 유실될 수 있기 떄문에 컬럼 이름을 명시적으로 붙이는 것이 좋다.

In [18]:
# 인덱스 재설정 메소드 : reindex
# 인덱스 초기화 메소드 : reset_index

df_all_email = df_email.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()

In [19]:
df_all_email.columns = ['week', 'user', 'emailsOpened']
df_all_email.head(20)

,week,user,emailsOpened
0,2016-07-11,1,3
1,2016-07-11,3,0
2,2016-07-11,5,0
3,2016-07-11,6,0
4,2016-07-11,9,0
5,2016-07-11,10,0
6,2016-07-11,14,1
7,2016-07-11,16,0
8,2016-07-11,20,0
9,2016-07-11,21,0


In [20]:
# 998번 회원 다시보기: 0인 주가 나타났다.

df_all_email[df_all_email['user'] == 998].sort_values('week')

,week,user,emailsOpened
43658,2015-02-09,998,0
1616,2015-02-16,998,0
71147,2015-02-23,998,0
79771,2015-03-02,998,0
44736,2015-03-09,998,0
...,...,...,...
50126,2018-04-30,998,3
53899,2018-05-07,998,3
2155,2018-05-14,998,3
5389,2018-05-21,998,3


특정 유저가 최초로 이메일을 열람하기 전과 같은 데이터를 null로 유지할 수 있는 방법이 사실 많지 않다. 최초 이메일 수신 시점 데이터라도 있으면 수월하겠지만, 현재 데이터로는 여기까지가 최선이다. 

### 기록의 시작과 끝 결정 및 불필요한 기간 삭제

1. 이메일 데이터프레임을 유저별로 그룹화하여 start_date와 end_date를 기획하고, 이로부터 수신해온 주의 시작과 끝(min과 max)를 얻을 수 있다.



In [21]:
cutoff_dates = df_email.groupby('user').week.agg(['min', 'max']).reset_index()

In [22]:
cutoff_dates

,user,min,max
0,1,2015-06-29,2018-05-28
1,3,2018-03-05,2018-04-23
2,5,2017-06-05,2018-05-28
3,6,2016-12-05,2018-05-28
4,9,2016-07-18,2018-05-28
...,...,...,...
534,991,2016-10-24,2016-10-24
535,992,2015-02-09,2015-07-06
536,993,2017-09-11,2018-05-28
537,995,2016-09-05,2018-05-28


In [23]:
cutoff_dates = cutoff_dates.reset_index()

2. 이 데이터프레임으로 0이었던 열처럼 필요가 없는 것으로 간주하는 행들은 삭제하도록 한다.  
0이 아닌 값이 나타나기 전의 시점들을 행 삭제 대상으로 지정한다.

해당 max나 min날짜가 같이 지워지면 안되므로 <, >을 사용한다.

In [24]:
for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    df_all_email.drop(df_all_email[df_all_email['user'] == user ][df_all_email['week'] < start_date].index, inplace = True)
    df_all_email.drop(df_all_email[df_all_email['user'] == user ][df_all_email['week'] > end_date].index, inplace = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_84720/3016188976.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_all_email.drop(df_all_email[df_all_email['user'] == user ][df_all_email['week'] < start_date].index, inplace = True)
/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_84720/3016188976.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_all_email.drop(df_all_email[df_all_email['user'] == user ][df_all_email['week'] > end_date].index, inplace = True)


In [25]:
df_all_email

,week,user,emailsOpened
0,2016-07-11,1,3
6,2016-07-11,14,1
12,2016-07-11,28,0
13,2016-07-11,31,3
14,2016-07-11,33,3
...,...,...,...
93215,2015-04-13,929,2
93216,2015-04-13,932,2
93221,2015-04-13,940,0
93238,2015-04-13,977,0


시간을 인식할 수 있는 형태 특히, 타임스탬프의 생성방식과 그 의미에 따라 데이터를 저장해야 한다. 필요하다면 데이터 엔지니어와 데이터베이스 운영자를 설득해야 할 수도 있다. 문제 해결의 전체 과정 중 이전 공정이 많은 문제를 해결할 수록 다음 공정에서 수행해야 하는 작업은 줄어들기 마련.

### 요약
email data를 정리해보았다. 이때 고려할 수 있는 질문들은 다음과 같다.

1. 기록된 데이터를 주 단위로 기부금을 취합하면 이메일 데이터와 기간 단위의 비교가 가능해진다. 따라서 회원의 이메일 응답과 기부가 어떤 식으로 연관이 있는지 합리적으로 조사할 수 있다.  
(예를 들어 몇 회 이상 조회된 주간의 기부금은 어느 정도 상승한다, 혹은 조회 후 평균적으로 기부라는 반응이 오기까지 얼마의 리드타임이 존재한다-같은 것을 이야기 하는 것 같다.
2. 이메일 개수를 주어진 주의 기부금의 예측변수로 취급할 수 있다. 전주를 이용하는 이유는 해당 주의 요약이고, 그 날짜까지 수집된 것을 기반으로 하기 때문이다. 반대로 기부를 한 다음 주에 기부자가 이메일을 열람하는지에 대한 여부도 예시가 될 수 있다.


## 2.2.2 발견된 시계열 구성하기

기부 데이터를 다운샘플링(주 단위로 묶어)하여 주간 시계열로 바꿀 수 있다. 그렇게 되면 이메일 데이터와 서로 비교할 수 있게 된다. 

타임스탬프를 주 단위로 취합한 다음 총계를 구해볼 수 있다. 주간 기부금은 기부자 개별의 기부 금액이 반영된 경우가 대부분이다. (한 주에 두 번 이상 기부할 가능성이 적기 때문)



In [26]:
# donation df의 시간 컬럼을 to_datetime해두어야 한다. 
df_dona.set_index('timestamp', inplace = True)
agg_dona = df_dona.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

In [27]:
agg_dona

user  timestamp 
0     2015-03-30      25.0
      2015-04-06       0.0
      2015-04-13       0.0
      2015-04-20       0.0
      2015-04-27       0.0
                     ...  
995   2017-09-11       0.0
      2017-09-18       0.0
      2017-09-25       0.0
      2017-10-02    1000.0
998   2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

여기부터는 변수명이 없던 것이 등장하고 하면서 오류가 나서 진행을 중단했다.  
말그대로 donation과 email open 횟수를 함께 한 데이터 프페임에 나타내도록 한 것은 알겠으나...

In [ ]:
for user, emailsOpened in df_all_email.groupby('user'):
    # 특정 회원의 기부 데이터 추출
    user_donations = agg_dona[agg_dona['user'] == user]   #user에서 오류남 어쩌라는 것임 user는 user인데...........
    
    # 기부 데이터의 인덱스를 timestamp로 설정
    user_donations.set_index('timestamp', inplace = True)
    df_all_email.set_index('week', inplace = True)
    
    df_all_email = df_all_email[df_all_email['user'] == user]
    df_all_email.sort_values('week').set_index('week')
    
    df = pd.merge(df_all_email, user_donations, how = 'left', left_index = True, right_index = True)
    df.fiilna(0)
    
    df['user'] = df.user_x
    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [ ]:
# 특정 기부 내역을 한 주 뒤로 옮기고 싶다면 shift를 사용할 수 있다. 
df = merged_df[merged_df['user']==998]
df['target'] = df.amout.shift(1)
df = df.fillna(0)
df

# 교재에 언급된 코드: 데이터셋 생성

In [ ]:
### code to generate the data set
import pdb
import numpy as np
import pandas as pd

## membership status
years      = ['2014', '2015', '2016', '2017', '2018']
userStatus = ['bronze', 'silver', 'gold', 'inactive']

userYears = np.random.choice(years, 1000, p = [0.1, 0.1, 0.15, 0.30, 0.35])
userStats = np.random.choice(userStatus, 1000, p = [0.5, 0.3, 0.1, 0.1])

yearJoined = pd.DataFrame({'yearJoined': userYears,
                           'userStats': userStats})

## email behavior
NUM_EMAILS_SENT_WEEKLY = 3

## types of behavior
def never_opens(period_rng):
    return []

def constant_open_rate(period_rng):
    n, p = NUM_EMAILS_SENT_WEEKLY, np.random.uniform(0, 1)
    num_opened = np.random.binomial(n, p, len(period_rng))
    return num_opened

def open_rate_with_factor_change(period_rng, fac):

    if len(period_rng) < 1 :
        return []
    
    times = np.random.randint(0, len(period_rng), int(0.1 * len(period_rng)))
    try:
        n, p = NUM_EMAILS_SENT_WEEKLY, np.random.uniform(0, 1)
        num_opened = np.zeros(len(period_rng))
        for pd in range(0, len(period_rng), 2):        
            num_opened[pd:(pd + 2)] = np.random.binomial(n, p, 2)
            p = max(min(1, p * fac), 0)
    except:
        num_opened[pd] = np.random.binomial(n, p, 1)
    for t in times:
        num_opened[t] = 0
    return num_opened

def increasing_open_rate(period_rng):
    return open_rate_with_factor_change(period_rng, np.random.uniform(1.01, 1.30))
    
def decreasing_open_rate(period_rng):
    return open_rate_with_factor_change(period_rng, np.random.uniform(0.5, 0.99))

def random_weekly_time_delta():
    days_of_week = [d for d in range(7)]
    hours_of_day = [h for h in range(11, 23)]
    minute_of_hour = [m for m in range(60)]
    second_of_minute = [s for s in range(60)]
    return pd.Timedelta(str(np.random.choice(days_of_week))     + " days" )   + \
           pd.Timedelta(str(np.random.choice(hours_of_day))     + " hours" )  + \
           pd.Timedelta(str(np.random.choice(minute_of_hour))   + " minutes") + \
           pd.Timedelta(str(np.random.choice(second_of_minute)) + " seconds")
    
## donation behavior
def produce_donations(period_rng, user_behavior, num_emails, use_id, user_join_year):
    donation_amounts = np.array([0, 25, 50, 75, 100, 250, 500, 1000, 1500, 2000])
    user_has = np.random.choice(donation_amounts)
        
    user_gives = num_emails  / (NUM_EMAILS_SENT_WEEKLY * len(period_rng)) * user_has
    user_gives_idx = np.where(user_gives >= donation_amounts)[0][-1]
    user_gives_idx = max(min(user_gives_idx, len(donation_amounts) - 2), 1)
    
    num_times_gave = np.random.poisson(2) * (2018 - user_join_year)
    
    times = np.random.randint(0, len(period_rng), num_times_gave)

    donations = pd.DataFrame({'user': [], 'amount': [], 'timestamp': []})
    for n in range(num_times_gave):
        
        donations = donations.append(pd.DataFrame({'user': [use_id],
                                       'amount': [donation_amounts[user_gives_idx + \
                                                  np.random.binomial(1, .3)]], 
                                       'timestamp': [str(period_rng[times[n]].start_time + random_weekly_time_delta())]}))

    if donations.shape[0] > 0:
        donations = donations[donations.amount != 0]
    return donations
    

## run it
behaviors = [never_opens, constant_open_rate, increasing_open_rate, decreasing_open_rate]
user_behaviors = np.random.choice(behaviors, 1000, [0.2, 0.5, 0.1, 0.2])
                                        
rng = pd.period_range('2015-02-14', '2018-06-01', freq = 'W')
emails = pd.DataFrame({'user': [], 'week': [], 'emailsOpened':[]})
donations = pd.DataFrame({'user': [], 'amount': [], 'timestamp': []})

for idx in range(yearJoined.shape[0]):
    ## randomly generate the date when a user would have joined
    join_date = pd.Timestamp(yearJoined.iloc[idx].yearJoined) + \
                pd.Timedelta(str(np.random.randint(0, 365)) + ' days')
    join_date = min(join_date, pd.Timestamp('2018-06-01'))
    
    ## user should not receive emails or make donations before joining
    ## thank you to Murray M Gillin for reporting errata and suggesting this correction
    user_rng = rng[rng.start_time > join_date]    
    
    if len(user_rng) < 1:
        continue

    info = user_behaviors[idx](user_rng)

    if len(info) == len(user_rng):
        emails = emails.append(pd.DataFrame({'user': [idx] * len(info),
                                    'week': [str(r.start_time) for r in user_rng],
                                    'emailsOpened': info}))

    donations = donations.append(produce_donations(user_rng, user_behaviors[idx],
                                                    sum(info), idx, join_date.year))



## get rid of zero donations and zero emails
emails = emails[emails.emailsOpened != 0]
yearJoined.index.name = 'user'

yearJoined.to_csv('data/year_joined.csv', index = False)
donations.to_csv( 'data/donations.csv',   index = False)
emails.to_csv(    'data/emails.csv',      index = False)


